In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import cv2
import math
import numpy as np
from skimage import measure
from skimage.measure import regionprops
from skimage.morphology import opening, erosion, dilation, disk
import matplotlib.pyplot as plt

In [0]:
TRAIN = ['230', '232', '242', '263']
TEST  = [ '221','229', '239', '246', '252']

In [0]:
TRAIN_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Ki67/LadderNet/Train/"
TEST_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Ki67/LadderNet/Test/"
MASK_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Ki67/Masks/"

In [0]:
def updateConfusionMatrix(ConfusionMatrix, PredictedImage, GroundTruth, PatientId, axis ):
    # Opening operation
    selem = disk(20)
    GroundTruth = dilation(GroundTruth,selem)
    GroundTruthLabel = measure.label(GroundTruth)

    rows,cols = PredictedImage.shape

    #  0 is background
    for label in range(1, GroundTruthLabel.max()+1):
      masklabel = np.zeros([rows,cols])
      masklabel[GroundTruthLabel==label] = 1

      resultantimage = cv2.multiply(PredictedImage, masklabel)

      areaGT = np.sum(masklabel)
      areaSM = np.sum(resultantimage)

      if(areaSM > 0 ):
        ConfusionMatrix[axis][0] = ConfusionMatrix[axis][0] + 1       #TP
        PredictedImage[GroundTruthLabel==label] = 0
      else:
        ConfusionMatrix[axis][2] = ConfusionMatrix[axis][2] + 1       #FN
    
    # PredictedImage = erosion(PredictedImage, selem)
    predictedprops = regionprops(measure.label(PredictedImage))

    for i in range(len(predictedprops)):
      if(predictedprops[i].area >= 5500):
        ConfusionMatrix[axis][1] += 1                                #FP

In [0]:
def getPatientWiseCount(ConfusionMatrix, PredictedPatient, GroundTruthPatient, PatientId):

  for imagenum in range(PredictedPatient.shape[0]):
  
    PredictedImage = PredictedPatient[imagenum]
    GroundTruth    = GroundTruthPatient[imagenum]
    rows,cols = PredictedImage.shape

    # dividing the image into strong,weak,intermediate
    predictedIP = np.zeros([rows,cols])
    predictedIN   = np.zeros([rows,cols])
    
    gtIP = np.zeros([rows,cols])
    gtIN = np.zeros([rows,cols])
    
    predictedIP[PredictedImage==255] = 1
    predictedIN[PredictedImage==128] = 1

    gtIP[GroundTruth==255] = 1
    gtIN[GroundTruth==128] = 1
    
    # 0th axis in confusion matrix is immunopositive
    # 1st axis in confusion matrix is immunonegative

    updateConfusionMatrix(ConfusionMatrix, predictedIP, gtIP, PatientId, 0)
    updateConfusionMatrix(ConfusionMatrix, predictedIN, gtIN, PatientId, 1)

**SM = Segmented Mask**

**GT = Ground Truth** 

In [24]:
train_confusion_matrix = np.zeros([ 2, 3])
test_confusion_matrix  = np.zeros([ 2, 3])

for patientid in range(len(TRAIN)):
  patientSM = np.load(TRAIN_PATH + TRAIN[patientid] + '.npy')
  patientGT      = np.load(MASK_PATH + 'IHC ' + TRAIN[patientid] + '.npy')
  getPatientWiseCount(train_confusion_matrix, patientSM, patientGT, patientid)
  print(TRAIN[patientid] + ' done.')
  print(train_confusion_matrix)

230 done.
[[219.   0.   2.]
 [328.   0.   8.]]
232 done.
[[276.  70. 186.]
 [564. 142. 125.]]
242 done.
[[721.  71. 188.]
 [921. 142. 132.]]
263 done.
[[ 771.  244.  264.]
 [1059.  291.  165.]]


In [25]:
for patientid in range(len(TEST)):
  patientSM = np.load(TEST_PATH + TEST[patientid] + '.npy')
  patientGT      = np.load(MASK_PATH + 'IHC ' + TEST[patientid] + '.npy')
  getPatientWiseCount(test_confusion_matrix, patientSM, patientGT, patientid)
  print(TEST[patientid] + ' done.')
  print(test_confusion_matrix)

221 done.
[[194.  33.  22.]
 [376.  14.   2.]]
229 done.
[[495.  34.  35.]
 [637.  15.  56.]]
239 done.
[[715.  36.  37.]
 [928.  32.  56.]]
246 done.
[[1019.   36.   47.]
 [1308.   54.   63.]]
252 done.
[[1158.   36.   54.]
 [1661.   57.   66.]]


In [0]:
def getMetrics(ConfusionMatrix, axis):
  precision = ConfusionMatrix[axis][0] / (ConfusionMatrix[axis][0] + ConfusionMatrix[axis][1] )
  recall    = ConfusionMatrix[axis][0] / (ConfusionMatrix[axis][0] + ConfusionMatrix[axis][2] )
  dice      = (2 * precision * recall) / (precision + recall)
  return precision, recall, dice

In [27]:
ptrain_precision, ptrain_recall, ptrain_dice = getMetrics(train_confusion_matrix, 0)
ntrain_precision, ntrain_recall, ntrain_dice = getMetrics(train_confusion_matrix, 1)

print('TRAIN :\n\n')
print("CLASS\t\tPRECISION\t\tRECALL\t\t\tDICE")
print('Immunopositive\t' + str(ptrain_precision) + '\t' + str(ptrain_recall) + '\t' + str(ptrain_dice))
print('Immunonegative\t' + str(ntrain_precision) +  '\t' + str(ntrain_recall) + '\t' + str(ntrain_dice))

TRAIN :


CLASS		PRECISION		RECALL			DICE
Immunopositive	0.7596059113300493	0.744927536231884	0.7521951219512196
Immunonegative	0.7844444444444445	0.8651960784313726	0.8228438228438228


In [28]:
ptest_precision, ptest_recall, ptest_dice = getMetrics(test_confusion_matrix, 0)
ntest_precision, ntest_recall, ntest_dice = getMetrics(test_confusion_matrix, 1)

print('TEST :\n\n')
print("CLASS\t\tPRECISION\t\tRECALL\t\t\tDICE")
print('Immunopositive\t' + str(ptest_precision) + '\t' + str(ptest_recall) + '\t' + str(ptest_dice))
print('Immunonegative\t' + str(ntest_precision) +  '\t' + str(ntest_recall) + '\t' + str(ntest_dice))

TEST :


CLASS		PRECISION		RECALL			DICE
Immunopositive	0.9698492462311558	0.9554455445544554	0.9625935162094762
Immunonegative	0.9668218859138533	0.9617834394904459	0.9642960812772133
